In [43]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
import ujson as json
import os
from tqdm import tqdm_notebook


IDs: MB 307; TK 337 \
https://docs.google.com/spreadsheets/d/15e1K0tg5ponA5R6YQkZfihrShTDLAKf5qeKaoVCiuhQ/edit#gid=0

In [2]:
#Team name, using IDs above: mlcourse_ai_fall2019_team_<id1>_…<id4> 

In [21]:
train_X = pd.read_csv('new_feat_train_small.csv', index_col='match_id_hash')
test_X = pd.read_csv('new_feat_test_small.csv', index_col='match_id_hash')
target =  pd.read_csv('/Users/m/Insight/MLcourse/Dota/targets_plus.csv', index_col='match_id_hash')
y = target['radiant_win']
full_df = train_X.merge(pd.DataFrame(y),how='outer',left_index=True,right_index=True)
train_df = pd.read_csv('/Users/m/Insight/MLcourse/Dota/train_features.csv', index_col='match_id_hash')
test_df = pd.read_csv('/Users/m/Insight/MLcourse/Dota/test_features.csv', index_col='match_id_hash')
train_full = train_df.merge(pd.DataFrame(y),how='outer',left_index=True,right_index=True)

In [38]:
#hero id columns names
ls_r_hero_id = ['r{}_hero_id'.format(i) for i in range(1,6)]
ls_d_hero_id = ['d{}_hero_id'.format(i) for i in range(1,6)]
ls_hero_id = ls_r_hero_id + ls_d_hero_id
#sub data frame of hero ids and target
hero_ids = train_full[ls_hero_id +['radiant_win']]
hero_ids_rad_win = hero_ids[hero_ids['radiant_win'] == True] #rad wins
hero_ids_rad_lose = hero_ids[hero_ids['radiant_win'] == False] #rad loses
winning_hero_ids1 = hero_ids_rad_win[ls_r_hero_id]
winning_hero_ids2 = hero_ids_rad_lose[ls_d_hero_id]
losing_hero_ids1 = hero_ids_rad_win[ls_d_hero_id]
losing_hero_ids2 = hero_ids_rad_lose[ls_r_hero_id]
winning_hero_ids1.rename(columns = {'r1_hero_id':'1_id', 'r2_hero_id':'2_id', 
                              'r3_hero_id':'3_id','r4_hero_id':'4_id',
                                    'r5_hero_id':'5_id'}, inplace = True)
winning_hero_ids2.rename(columns = {'d1_hero_id':'1_id', 'd2_hero_id':'2_id', 
                              'd3_hero_id':'3_id','d4_hero_id':'4_id',
                                    'd5_hero_id':'5_id'}, inplace = True)
losing_hero_ids1.rename(columns = {'d1_hero_id':'1_id', 'd2_hero_id':'2_id', 
                              'd3_hero_id':'3_id','d4_hero_id':'4_id',
                                    'd5_hero_id':'5_id'}, inplace = True)
losing_hero_ids2.rename(columns = {'r1_hero_id':'1_id', 'r2_hero_id':'2_id', 
                              'r3_hero_id':'3_id','r4_hero_id':'4_id',
                                    'r5_hero_id':'5_id'}, inplace = True)
#for all games, df of winner's hero ids only
winning_hero_ids = pd.concat([winning_hero_ids1, winning_hero_ids2], axis=0)
#for all games, df of loser's hero ids only
losing_hero_ids = pd.concat([losing_hero_ids1, losing_hero_ids2], axis=0)

#by hero, in how many games did the hero win / lose
winning_hero_counts = winning_hero_ids['1_id'].value_counts().sort_index() + winning_hero_ids['2_id'].value_counts().sort_index() + winning_hero_ids['3_id'].value_counts().sort_index() + winning_hero_ids['4_id'].value_counts().sort_index() + winning_hero_ids['5_id'].value_counts().sort_index()
losing_hero_counts = losing_hero_ids['1_id'].value_counts().sort_index() + losing_hero_ids['2_id'].value_counts().sort_index() + losing_hero_ids['3_id'].value_counts().sort_index() + losing_hero_ids['4_id'].value_counts().sort_index() + losing_hero_ids['5_id'].value_counts().sort_index()

#to dictionary, key is hero id, value is win / loss count
winning_hero_counts = winning_hero_counts.sort_values()
winning_hero_dict = winning_hero_counts.to_dict()
losing_hero_counts = losing_hero_counts.sort_values()
losing_hero_dict = losing_hero_counts.to_dict()

#now subtract wins - loses by hero (this will be one feature)
hero_counts_win_minus_lose = winning_hero_counts.sort_index() - losing_hero_counts.sort_index()
diff_hero_dict = hero_counts_win_minus_lose.to_dict()

#normalize by dividing by total number of games played (this is another feature)
from collections import Counter
total_games_dict = Counter(winning_hero_dict) + Counter(losing_hero_dict)
hero_id_normalize_dict = {k: (diff_hero_dict[k] / total_games_dict[k]) for k in diff_hero_dict}

#add the two new features to the data frame
for col in ls_hero_id:
    train_full[col+'success'] = train_full[col].map(diff_hero_dict)
    train_df[col+'success'] = train_df[col].map(diff_hero_dict)
    test_df[col+'success'] = test_df[col].map(diff_hero_dict)
for col in ls_hero_id:
    train_full[col+'norm'] = train_full[col].map(hero_id_normalize_dict)
    train_df[col+'norm'] = train_df[col].map(hero_id_normalize_dict)
    test_df[col+'norm'] = test_df[col].map(hero_id_normalize_dict)